# Finetuning LLaMa + Text-to-SQL 

This walkthrough shows you how to fine-tune LLaMa-7B on a Text-to-SQL dataset, and then use it for inference against
any database of structured data using LlamaIndex.

**NOTE**: This code is taken and adapted from Modal's `doppel-bot` repo: https://github.com/modal-labs/doppel-bot.
**NOTE**: A lot of the code is contained in the underlying Python scripts. We definitely encourage you to go and take a look!

### Load Training Data for Finetuning LLaMa

We load data from `b-mc2/sql-create-context` on Hugging Face: https://huggingface.co/datasets/b-mc2/sql-create-context.

This dataset consists of tuples of natural language queries, create table statements, and ground-truth SQL queries. This is the dataset that we use to finetune our SQL model.

In [1]:
from src.load_data_sql import load_data_sql

load_data_sql(data_dir="data_sql")

### Run Finetuning Script

We run our finetuning script on the loaded dataset.
The finetuning script contains the following components:
- We split the dataset into training and validation splits.
- We format each split into input/output tuples of token id's. This means that the labels are the same as inputs (loss signal is measured on full input, not just on the generated portion). 
- We use `LoraConfig` from `peft` for efficient fine-tuning.
- We use `transformers.Trainer` to actually run the training process.
- If a valid `WANDB_PROJECT` is specified, along with the relevant secret in Modal, then we will log results to wandb.

We use Modal to spin up an A100 to run our finetuning code. 

In [ ]:
from src.finetune_sql import finetune

finetune(data_dir="data_sql")

### Integrate Model with LlamaIndex

Now that the model is 